In [3]:
import os
import time
import logging
import yaml
import ast
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
import torch
from torch_sparse import SparseTensor
from pprgo import utils, ppr
from pprgo.pprgo import PPRGo, RobustPPRGo
from pprgo.train import train
from pprgo.predict import predict_power_iter, predict_batched
from pprgo.dataset import PPRDataset, RobustPPRDataset
from pprgo.pytorch_utils import matrix_to_torch

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
model_type = "RobustPPRGO"

In [6]:
# choose dataset class
if model_type == "RobustPPRGO":
    DatasetClass = RobustPPRDataset
    ModelClass = RobustPPRGo
else:
    DatasetClass = PPRDataset
    ModelClass = PPRGo


In [7]:
# Set up logging
logger = logging.getLogger()
logger.handlers = []
ch = logging.StreamHandler()
formatter = logging.Formatter(
        fmt='%(asctime)s (%(levelname)s): %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S')
ch.setFormatter(formatter)
logger.addHandler(ch)
logger.setLevel('INFO')

# Download dataset

In [8]:
#!wget --show-progress -O data/reddit.npz https://ndownloader.figshare.com/files/23742119

# Load config

In [9]:
with open('config_demo.yaml', 'r') as c:
    config = yaml.safe_load(c)

In [10]:
# For strings that yaml doesn't parse (e.g. None)
for key, val in config.items():
    if type(val) is str:
        try:
            config[key] = ast.literal_eval(val)
        except (ValueError, SyntaxError):
            pass

In [11]:
data_file           = "/nfs/shared/data/mag_large_filtered_06_09_fine_standardized.npz"           # Path to the .npz data file
split_seed          = config['split_seed']        # Seed for splitting the dataset into train/val/test
ntrain_div_classes  = config['ntrain_div_classes']  # Number of training nodes divided by number of classes
attr_normalization  = config['attr_normalization']  # Attribute normalization. Not used in the paper

alpha               = config['alpha']               # PPR teleport probability
eps                 = config['eps']                 # Stopping threshold for ACL's ApproximatePR
topk                = config['topk']                # Number of PPR neighbors for each node
ppr_normalization   = config['ppr_normalization']   # Adjacency matrix normalization for weighting neighbors

hidden_size         = config['hidden_size']         # Size of the MLP's hidden layer
nlayers             = config['nlayers']            # Number of MLP layers
weight_decay        = config['weight_decay']        # Weight decay used for training the MLP
dropout             = config['dropout']             # Dropout used for training

lr                  = config['lr']                  # Learning rate
max_epochs          = 10          # Maximum number of epochs (exact number if no early stopping)
batch_size          = 2048          # Batch size for training
batch_mult_val      = config['batch_mult_val']      # Multiplier for validation batch size
pred_batch_size     = 10000

eval_step           = config['eval_step']           # Accuracy is evaluated after every this number of steps
run_val             = config['run_val']             # Evaluate accuracy on validation set during training

early_stop          = config['early_stop']          # Use early stopping
patience            = config['patience']            # Patience for early stopping

nprop_inference     = config['nprop_inference']     # Number of propagation steps during inference
inf_fraction        = 0.2        # Fraction of nodes for which local predictions are computed during inference

# Load the data

In [1]:
start = time.time()
(adj_matrix, attr_matrix, labels,
 train_idx, val_idx, test_idx) = utils.get_data(
        f"{data_file}",
        seed=split_seed,
        ntrain_div_classes=ntrain_div_classes,
        normalize_attr=attr_normalization
)
try:
    d = attr_matrix.n_columns
except AttributeError:
    d = attr_matrix.shape[1]
nc = labels.max() + 1
time_loading = time.time() - start
print(f"Runtime: {time_loading:.2f}s")

NameError: name 'time' is not defined

In [ ]:
len(train_idx)

# Preprocessing: Calculate PPR scores

In [ ]:
# compute the ppr vectors for train/val nodes using ACL's ApproximatePR
start = time.time()
topk_train = ppr.topk_ppr_matrix(adj_matrix, alpha, eps, train_idx, topk,
                                 normalization=ppr_normalization)
train_set = DatasetClass(attr_matrix_all=attr_matrix, ppr_matrix=topk_train, indices=train_idx, labels_all=labels)
if run_val:
    topk_val = ppr.topk_ppr_matrix(adj_matrix, alpha, eps, val_idx, topk,
                                   normalization=ppr_normalization)
    val_set = DatasetClass(attr_matrix_all=attr_matrix, ppr_matrix=topk_val, indices=val_idx, labels_all=labels)
else:
    val_set = None
time_preprocessing = time.time() - start
print(f"Runtime: {time_preprocessing:.2f}s")

In [ ]:
len(train_set)

In [ ]:
train_loader = torch.utils.data.DataLoader(
        dataset=train_set,
        sampler=torch.utils.data.BatchSampler(
            torch.utils.data.SequentialSampler(train_set),
            batch_size=64, drop_last=False
        ),
        batch_size=None,
        num_workers=0,
    )

# Training: Set up model and train

In [ ]:
start = time.time()
model = ModelClass(d, nc, hidden_size, nlayers, dropout, aggr="mean")
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

nepochs, _, _ = train(
        model=model, train_set=train_set, val_set=val_set,
        lr=lr, weight_decay=weight_decay,
        max_epochs=max_epochs, batch_size=batch_size, batch_mult_val=batch_mult_val,
        eval_step=eval_step, early_stop=early_stop, patience=patience)
time_training = time.time() - start
logging.info('Training done.')
print(f"Runtime: {time_training:.2f}s")

# Inference (val and test)

In [ ]:
start = time.time()
predictions, time_logits, time_propagation = predict(
        model=model, adj_matrix=adj_matrix, attr_matrix=attr_matrix, alpha=alpha,
        nprop=nprop_inference, inf_fraction=0.0001,
        ppr_normalization=ppr_normalization,
        batch_size_logits=512)
time_inference = time.time() - start
print(f"Runtime: {time_inference:.2f}s")

2021-01-04 17:50:33 (INFO): Prediction step 0/2 - 0.00%
2021-01-04 17:50:33 (INFO): Prediction step 1/2 - 0.41%
2021-01-04 17:50:33 (INFO): Prediction step 2/2 - 0.83%
2021-01-04 17:50:33 (INFO): Calculated all logits
2021-01-04 17:50:33 (INFO): 3
2021-01-04 17:50:33 (INFO): 512


# Collect and print results

In [ ]:
acc_train = 100 * accuracy_score(labels[train_idx], predictions[train_idx])
acc_val = 100 * accuracy_score(labels[val_idx], predictions[val_idx])
acc_test = 100 * accuracy_score(labels[test_idx], predictions[test_idx])
f1_train = f1_score(labels[train_idx], predictions[train_idx], average='macro')
f1_val = f1_score(labels[val_idx], predictions[val_idx], average='macro')
f1_test = f1_score(labels[test_idx], predictions[test_idx], average='macro')

gpu_memory = torch.cuda.max_memory_allocated()
memory = utils.get_max_memory_bytes()

time_total = time_preprocessing + time_training + time_inference

In [ ]:
print(f'''
Accuracy: Train: {acc_train:.1f}%, val: {acc_val:.1f}%, test: {acc_test:.1f}%
F1 score: Train: {f1_train:.3f}, val: {f1_val:.3f}, test: {f1_test:.3f}

Runtime: Preprocessing: {time_preprocessing:.2f}s, training: {time_training:.2f}s, inference: {time_inference:.2f}s -> total: {time_total:.2f}s
Memory: Main: {memory / 2**30:.2f}GB, GPU: {gpu_memory / 2**30:.3f}GB
''')